<a href="https://colab.research.google.com/github/tomtyiu/SentinelShieldAI/blob/main/Speech_to_Groq_Shield.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00


In [ ]:
!apt install python3.10-venv
!python -m venv env1

!source env1/bin/activate

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done


In [2]:
import pyaudio
import wave
import os
from groq import Groq

FORMAT = pyaudio.paInt16
CHANNELS = 2
RATE = 16000
CHUNK = 1024
RECORD_SECONDS = 15
WAVE_OUTPUT_FILENAME = "file.wav"

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
    )


def recording(WAVE_OUTPUT_FILENAME):
    audio = pyaudio.PyAudio()

    # start Recording
    stream = audio.open(format=FORMAT, channels=CHANNELS,
                    rate=RATE, input=True,
                    frames_per_buffer=CHUNK)
    print("recording...")
    frames = []

    for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
        data = stream.read(CHUNK)
        frames.append(data)
    print("finished recording")


    # stop Recording
    stream.stop_stream()
    stream.close()
    audio.terminate()

    waveFile = wave.open(WAVE_OUTPUT_FILENAME, 'wb')
    waveFile.setnchannels(CHANNELS)
    waveFile.setsampwidth(audio.get_sample_size(FORMAT))
    waveFile.setframerate(RATE)
    waveFile.writeframes(b''.join(frames))
    waveFile.close()


ModuleNotFoundError: No module named 'pyaudio'

In [ ]:
def transcribe(WAVE_OUTPUT_FILENAME):
    with open(WAVE_OUTPUT_FILENAME, "rb") as file:
        transcription = client.audio.transcriptions.create(
        file=(WAVE_OUTPUT_FILENAME, file.read()),
        model="whisper-large-v3",
        prompt="Specify context or spelling",  # Optional
        response_format="json",  # Optional
        language="en",  # Optional
        temperature=0.0  # Optional
        )
        print(transcription.text)
        return transcription.text

In [ ]:
from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import AutoTokenizer, pipeline
from google.colab import userdata
HK_TOKEN=userdata.get('HK_TOKEN')

tokenizer = AutoTokenizer.from_pretrained("ProtectAI/deberta-v3-base-prompt-injection-v2", subfolder="onnx")
tokenizer.model_input_names = ["input_ids", "attention_mask"]
model = ORTModelForSequenceClassification.from_pretrained("ProtectAI/deberta-v3-base-prompt-injection-v2", export=False, subfolder="onnx")

classifier = pipeline(
  task="text-classification",
  model=model,
  tokenizer=tokenizer,
  truncation=True,
  max_length=512,
)



In [ ]:
from tenacity import retry, stop_after_attempt, wait_fixed, retry_if_exception_type
from groq import Groq
import time
from google.colab import userdata

@retry(
    wait=wait_fixed(2),
    stop=stop_after_attempt(3),  # Adjust the number of retry attempts as needed
    retry=retry_if_exception_type(Exception)  # Retry on any exception
)
def make_groq_request(input):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "user",
                    "content": input,
                    }
                ],
            model="llama3-8b-8192",
        )
        # Process the response
        return chat_completion
    except Exception as e:
        # Handle rate limit exceeded error
        if "Rate limit exceeded" in str(e):
            # Extract retry-after time from headers if available
            retry_after_tokens = float(e.response.headers.get('x-ratelimit-reset-tokens', 2))
            retry_after_requests = float(e.response.headers.get('x-ratelimit-reset-requests', 2))
            retry_after = max(retry_after_tokens, retry_after_requests)
            print(f"Rate limit exceeded. Retrying after {retry_after} seconds...")
            time.sleep(retry_after)
        raise e  # Re-raise the exception to trigger retry

# Make the request with retries
record=recording(WAVE_OUTPUT_FILENAME)
input=transcribe(WAVE_OUTPUT_FILENAME)
result=classifier(input)
response = make_groq_request(result)
print(response.choices[0].message.content)